# Imports

In [ ]:
import os
import pickle
import time, datetime

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision
from tqdm import tqdm
import tensorflow as tf

# Save Methods

In [ ]:
# Plot training loss
def show_train_hist(hist, show=False, save=False, path='Train_hist.png'):
    """Loss tracker
    
    Plot the losses of the networks independently to see the trend
    
    Arguments:
        hist {[dict]} -- Tracking variables
    
    Keyword Arguments:
        show {bool} -- If to display the figure (default: {False})
        save {bool} -- If to store the figure (default: {False})
        path {str} -- path to store the figure (default: {'Train_hist.png'})
    """
    x = range(len(hist['VGG_losses']))

    y1 = hist['VGG_losses']
    y2 = hist['ResNet_losses']
    y3 = hist['GGL_Net_losses']

    plt.plot(x, y1, label='VGG_loss')
    plt.plot(x, y2, label='ResNet_loss')
    plt.plot(x, y3, label='GoogLeNet_loss')

    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

# Plot validation accuracy
def show_accuracy_plot(hist, show=False, save=False, path='Accuracy_hist.png'):
    """Loss tracker
    
    Plot the losses of the networks independently to see the trend
    
    Arguments:
        hist {[dict]} -- Tracking variables
    
    Keyword Arguments:
        show {bool} -- If to display the figure (default: {False})
        save {bool} -- If to store the figure (default: {False})
        path {str} -- path to store the figure (default: {'Train_hist.png'})
    """
    x = range(len(hist['VGG_accuracy']))

    y1 = hist['VGG_accuracy']
    y2 = hist['ResNet_accuracy']
    y3 = hist['GGL_Net_accuracy']

    plt.plot(x, y1, label='VGG_Accuracy')
    plt.plot(x, y2, label='ResNet_Accuracy')
    plt.plot(x, y3, label='GoogLeNet_Accuracy')

    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')

    plt.legend(loc=4)
    plt.grid(True)
    #plt.tight_layout()

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

# Initial Set-up

In [ ]:
# initialise the device for training, if gpu is available, device = 'cuda', else: device = 'cpu'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

save_dir = './Results/'
model_dir = './Models/'
# create folder if not exist
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

cuda


# Model loading and training

## Create Models and Model Training Methods

### VGG Model methods

In [ ]:
def LoadVGG(device):
    VGG_net = torch.hub.load('pytorch/vision:v0.9.0', 'vgg19', pretrained=False)
    VGG_net = VGG_net.to(device)

    VGG_optimizer = optim.SGD(VGG_net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)
    VGG_scheduler = optim.lr_scheduler.ReduceLROnPlateau(VGG_optimizer, mode='max', patience=5, verbose=True)

    return VGG_net, VGG_optimizer, VGG_scheduler

def TrainVGG(VGG_net, optimizer, scheduler, criterion, epochs, train_loader, validation_loader):
    print('\nTraining VGG')
    train_hist['per_epoch_ptimes'].clear()
    start_time = time.time()
    now = datetime.datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print('Training starting ',str(dt_string))
    for epoch in range(epochs):
        Train_Loss = []
        epoch_start_time = time.time()
        now = datetime.datetime.now()
        print('Epoch %d starting at: %02d:%02d:%02d'%(epoch+1, now.hour, now.minute, now.second))
        VGG_net.train()
        # Training
        for (train_images, train_labels) in tqdm(train_loader):
            train_images = train_images.to(device)
            train_labels = train_labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            train_outputs = VGG_net(train_images)
            train_loss = criterion(train_outputs, train_labels)
            train_loss.backward()
            optimizer.step()

            Train_Loss.append(train_loss.item())
        
        VGG_net.eval()
        Val_Loss = []
        total = 0
        correct = 0
        # Validation
        for (val_images, val_labels) in tqdm(validation_loader):
            val_images = val_images.to(device)
            val_labels = val_labels.to(device)

            # calculate outputs by running images through the network
            val_outputs = VGG_net(val_images)
            val_loss = criterion(val_outputs, val_labels)
            
            Val_Loss.append(val_loss.item())

            # apply softmax function to get probabilities
            s = nn.Softmax(dim=1)
            probabilities = s(val_outputs)
            # class with highest probability is the predicted class
            _, predicted = torch.max(probabilities.data, 1)
            total += val_labels.size(0)
            correct += (predicted == val_labels).sum().item()

        # validation accuracy
        epoch_accuracy = 100 * correct / total

        # Learning Rate Adaptation step
        # changes LR on flat plateau
        scheduler.step(epoch_accuracy)

        epoch_loss = np.mean(Train_Loss)  # mean loss for the epoch
        epoch_end_time = time.time()
        per_epoch_ptime = epoch_end_time - epoch_start_time

        print("\nEpoch %d of %d, %.2f s, with average training loss: %.8f, and validation loss: %.8f, validation accuracy: %.2f" 
              % (epoch + 1, epochs, per_epoch_ptime, epoch_loss, np.mean(Val_Loss), epoch_accuracy))

        # record the loss for every epoch
        train_hist['VGG_losses'].append(epoch_loss)
        train_hist['VGG_accuracy'].append(epoch_accuracy)
        train_hist['per_epoch_ptimes'].append(per_epoch_ptime)

    now = datetime.datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print('Finished Training ',str(dt_string))

    end_time = time.time()
    total_ptime = end_time - start_time
    train_hist['total_ptime'].append(total_ptime)

    print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(train_hist['per_epoch_ptimes']), epochs, total_ptime))

### ResNet Model methods

In [ ]:
def LoadResNet(device):
    ResNet_net = torch.hub.load('pytorch/vision:v0.9.0', 'resnet34', pretrained=False)
    ResNet_net = ResNet_net.to(device)

    ResNet_optimizer = optim.SGD(ResNet_net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)
    ResNet_scheduler = optim.lr_scheduler.ReduceLROnPlateau(ResNet_optimizer, mode='max', patience=5, verbose=True)

    return ResNet_net, ResNet_optimizer, ResNet_scheduler

def TrainResNet(ResNet_net, optimizer, scheduler, criterion, epochs, train_loader, validation_loader):
    print('\nTraining ResNet')
    train_hist['per_epoch_ptimes'].clear()
    start_time = time.time()
    now = datetime.datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print('Training starting ',dt_string)
    for epoch in range(epochs):
        Train_Loss = []
        epoch_start_time = time.time()
        now = datetime.datetime.now()
        print('Epoch %d starting at: %02d:%02d:%02d'%(epoch+1, now.hour, now.minute, now.second))
        ResNet_net.train()
        # Training
        for (train_images, train_labels) in tqdm(train_loader):
            train_images = train_images.to(device)
            train_labels = train_labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            train_outputs = ResNet_net(train_images)
            train_loss = criterion(train_outputs, train_labels)
            train_loss.backward()
            optimizer.step()

            Train_Loss.append(train_loss.item())
        
        ResNet_net.eval()
        Val_Loss = []
        total = 0
        correct = 0
        # Validation
        for (val_images, val_labels) in tqdm(validation_loader):
            val_images = val_images.to(device)
            val_labels = val_labels.to(device)

            # calculate outputs by running images through the network
            val_outputs = ResNet_net(val_images)
            val_loss = criterion(val_outputs, val_labels)
            
            Val_Loss.append(val_loss.item())

            # apply softmax function to get probabilities
            s = nn.Softmax(dim=1)
            probabilities = s(val_outputs)
            # class with highest probability is the predicted class
            _, predicted = torch.max(probabilities.data, 1)
            total += val_labels.size(0)
            correct += (predicted == val_labels).sum().item()

        # validation accuracy
        epoch_accuracy = 100 * correct / total

        # Learning Rate Adaptation step
        # changes LR on flat plateau
        scheduler.step(epoch_accuracy)

        epoch_loss = np.mean(Train_Loss)  # mean generator loss for the epoch
        epoch_end_time = time.time()
        per_epoch_ptime = epoch_end_time - epoch_start_time

        print("\nEpoch %d of %d, %.2f s, with average training loss: %.8f, and validation loss: %.8f, validation accuracy: %.2f" 
              % (epoch + 1, epochs, per_epoch_ptime, epoch_loss, np.mean(Val_Loss), epoch_accuracy))

        # record the loss for every epoch
        train_hist['ResNet_losses'].append(epoch_loss)
        train_hist['ResNet_accuracy'].append(epoch_accuracy)
        train_hist['per_epoch_ptimes'].append(per_epoch_ptime)

    now = datetime.datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print('Finished Training ',str(dt_string))

    end_time = time.time()
    total_ptime = end_time - start_time
    train_hist['total_ptime'].append(total_ptime)

    print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(train_hist['per_epoch_ptimes']), epochs, total_ptime))

### GoogLeNet Model Methods

In [ ]:
def LoadGLNet(device):
    GLNet_net = torch.hub.load('pytorch/vision:v0.9.0', 'googlenet', pretrained=False)
    GLNet_net = GLNet_net.to(device)

    GLNet_optimizer = optim.SGD(GLNet_net.parameters(), lr=0.01, momentum=0.9)
    GLNet_scheduler = optim.lr_scheduler.StepLR(GLNet_optimizer, step_size=8, gamma=0.96, verbose=True)

    return GLNet_net, GLNet_optimizer, GLNet_scheduler

def TrainGLNet(GLNet_net, optimizer, scheduler, criterion, epochs, train_loader):
    print('\nTraining GoogLeNet')
    train_hist['per_epoch_ptimes'].clear()
    start_time = time.time()
    now = datetime.datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print('Training starting ',dt_string)
    for epoch in range(epochs):
        # Record epoch start time
        epoch_start_time = time.time()
        now = datetime.datetime.now()
        print('Epoch %d starting at: %02d:%02d:%02d'%(epoch+1, now.hour, now.minute, now.second))

        GLNet_net.train()
        Train_Loss = []
        # Training
        for (train_images, train_labels) in tqdm(train_loader):
            train_images = train_images.to(device)
            train_labels = train_labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            train_outputs = GLNet_net(train_images)
            train_outputs = train_outputs.logits
            train_loss = criterion(train_outputs, train_labels)
            train_loss.backward()
            optimizer.step()

            Train_Loss.append(train_loss.item())

        GLNet_net.eval()
        Val_Loss = []
        total = 0
        correct = 0
        # Validation
        for (val_images, val_labels) in tqdm(validation_loader):
            val_images = val_images.to(device)
            val_labels = val_labels.to(device)

            # calculate outputs by running images through the network
            val_outputs = GLNet_net(val_images)

            # apply softmax function to get probabilities
            s = nn.Softmax(dim=1)
            probabilities = s(val_outputs)
            # class with highest probability is the predicted class
            _, predicted = torch.max(probabilities.data, 1)
            total += val_labels.size(0)
            correct += (predicted == val_labels).sum().item()

        # validation accuracy
        epoch_accuracy = 100 * correct / total

        # Learning Rate Adaptation step
        # changes LR every 8 epochs
        scheduler.step()

        epoch_loss = np.mean(Train_Loss)  # mean generator loss for the epoch
        epoch_end_time = time.time()
        per_epoch_ptime = epoch_end_time - epoch_start_time

        print("\nEpoch %d of %d, %.2f s, with average training loss: %.8f, and validation loss: %.8f, validation accuracy: %.2f" 
              % (epoch + 1, epochs, per_epoch_ptime, epoch_loss, np.mean(Val_Loss), epoch_accuracy))

        # record the loss for every epoch
        train_hist['GGL_Net_losses'].append(epoch_loss)
        train_hist['GGL_Net_accuracy'].append(epoch_accuracy)
        train_hist['per_epoch_ptimes'].append(per_epoch_ptime)

    now = datetime.datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print('Finished Training ',str(dt_string))

    end_time = time.time()
    total_ptime = end_time - start_time
    train_hist['total_ptime'].append(total_ptime)

    print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(train_hist['per_epoch_ptimes']), epochs, total_ptime))

## Train on MNIST

### Data Loading and Processing

In [ ]:
data_dir = './MNIST_data/'

# training parameters
batch_size = 256
epochs = 100

# construct the datasets and data loaders
transform = transforms.Compose([transforms.Grayscale(3),
                                transforms.Resize(64),
                                transforms.RandomHorizontalFlip(0.5),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

tv_data = datasets.MNIST(root=data_dir, train=True, transform=transform, download=True)
train_data, validation_data = torch.utils.data.random_split(tv_data, [50000, 10000])

train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(dataset=validation_data, batch_size=batch_size, shuffle=True)

test_data = datasets.MNIST(root=data_dir, train=False, transform=transform, download=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

### Training Setup

In [ ]:
# tracking variables
train_hist = {}
train_hist['VGG_losses'] = []
train_hist['VGG_accuracy'] = []
train_hist['ResNet_losses'] = []
train_hist['ResNet_accuracy'] = []
train_hist['GGL_Net_losses'] = []
train_hist['GGL_Net_accuracy'] = []
train_hist['per_epoch_ptimes'] = []
train_hist['total_ptime'] = []

# Loss function
criterion = nn.CrossEntropyLoss().to(device)

# Classes
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

### VGG Training

In [ ]:
# Load and trian VGG
net, optimizer, scheduler = LoadVGG(device)
TrainVGG(net, optimizer, scheduler, criterion, epochs, train_loader, validation_loader)
torch.save(net.state_dict(), model_dir + '/VGG_MNIST_model.pth')
del net
del optimizer
del scheduler

### ResNet Training

In [ ]:
# Load and train ResNet
net, optimizer, scheduler = LoadResNet(device)
TrainResNet(net, optimizer, scheduler, criterion, epochs, train_loader, validation_loader)
torch.save(net.state_dict(), model_dir + '/ResNet_MNIST_model.pth')
del net
del optimizer
del scheduler

### GoogLeNet Training

In [ ]:
# Load and train GoogLeNet
net, optimizer, scheduler = LoadGLNet(device)
TrainGLNet(net, optimizer, scheduler, criterion, epochs, train_loader)
torch.save(net.state_dict(), model_dir + '/GoogLeNet_MNIST_model.pth')
del net
del optimizer
del scheduler

### Save training results

In [ ]:
# Show training results
show_train_hist(train_hist, save=True, path=save_dir + '/MNIST_train_hist.png')

# Add zeros at the beginning of data for graph 'smoothness'
train_hist['VGG_accuracy'].insert(0, 0)
train_hist['ResNet_accuracy'].insert(0, 0)
train_hist['GGL_Net_accuracy'].insert(0, 0)
# Show accuracy results
show_accuracy_plot(train_hist, save=True, path=save_dir + '/MNIST_accuracy_hist.png')

## Train on CIFAR10

### Data Loading and Processing

In [ ]:
data_dir = './CIFAR10_data/'

# training parameters
batch_size = 256
epochs = 100

# construct the datasets and data loaders
transform = transforms.Compose([transforms.Resize(64),
                                transforms.RandomHorizontalFlip(0.5),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
tv_data = datasets.CIFAR10(root=data_dir, train=True, transform=transform, download=True)
train_data, validation_data = torch.utils.data.random_split(tv_data, [40000, 10000])

train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(dataset=validation_data, batch_size=batch_size, shuffle=True)

test_data = datasets.CIFAR10(root=data_dir, train=False, transform=transform, download=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

Files already downloaded and verified


### Training Setup

In [ ]:
train_hist = {}
train_hist['VGG_losses'] = []
train_hist['VGG_accuracy'] = []
train_hist['ResNet_losses'] = []
train_hist['ResNet_accuracy'] = []
train_hist['GGL_Net_losses'] = []
train_hist['GGL_Net_accuracy'] = []
train_hist['per_epoch_ptimes'] = []
train_hist['total_ptime'] = []

# Loss function
criterion = nn.CrossEntropyLoss().to(device)

# Classes
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### VGG Training

In [ ]:
net, optimizer, scheduler = LoadVGG(device)
TrainVGG(net, optimizer, scheduler, criterion, epochs, train_loader, validation_loader)
torch.save(net.state_dict(), model_dir + '/VGG_CIFAR10_model.pth')
del net
del optimizer
del scheduler

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
  0%|          | 0/157 [00:00<?, ?it/s]


Training VGG
Training starting  10/05/2021 18:24:05
Epoch 1 starting at: 18:24:05


  0%|          | 0/157 [00:00<?, ?it/s]


Epoch 1 of 100, 42.19 s, with average training loss: 3.57973525, and validation loss: 2.32739629, validation accuracy: 9.91
Epoch 2 starting at: 18:24:47


  0%|          | 0/157 [00:00<?, ?it/s]

...


Epoch 98 of 100, 43.16 s, with average training loss: 0.00293140, and validation loss: 0.85819726, validation accuracy: 85.61
Epoch 99 starting at: 19:34:42


  0%|          | 0/157 [00:00<?, ?it/s]


Epoch 99 of 100, 43.10 s, with average training loss: 0.00327424, and validation loss: 0.83744534, validation accuracy: 85.41
Epoch 100 starting at: 19:35:25


100%|██████████| 40/40 [00:05<00:00,  7.42it/s]



Epoch 100 of 100, 43.09 s, with average training loss: 0.00321149, and validation loss: 0.81924972, validation accuracy: 85.75
Finished Training  10/05/2021 19:36:08
Avg per epoch ptime: 43.24, total 100 epochs ptime: 4323.53


### ResNet Training

In [ ]:
# Load and train ResNet
net, optimizer, scheduler = LoadResNet(device)
TrainResNet(net, optimizer, scheduler, criterion, epochs, train_loader, validation_loader)
torch.save(net.state_dict(), model_dir + '/ResNet_CIFAR10_model.pth')
del net
del optimizer
del scheduler

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
  1%|          | 1/157 [00:00<00:23,  6.68it/s]


Training ResNet
Training starting  10/05/2021 19:36:10
Epoch 1 starting at: 19:36:10


  1%|          | 1/157 [00:00<00:22,  6.92it/s]


Epoch 1 of 100, 27.21 s, with average training loss: 1.75723811, and validation loss: 1.52720597, validation accuracy: 46.09
Epoch 2 starting at: 19:36:37


  1%|          | 1/157 [00:00<00:24,  6.50it/s]

...


Epoch 98 of 100, 27.34 s, with average training loss: 0.00058561, and validation loss: 1.21767228, validation accuracy: 78.10
Epoch 99 starting at: 20:20:30


  1%|          | 1/157 [00:00<00:22,  6.91it/s]


Epoch 99 of 100, 27.31 s, with average training loss: 0.00057145, and validation loss: 1.20509507, validation accuracy: 78.42
Epoch 100 starting at: 20:20:57


100%|██████████| 40/40 [00:04<00:00,  9.16it/s]



Epoch 100 of 100, 27.13 s, with average training loss: 0.00056574, and validation loss: 1.25492236, validation accuracy: 77.75
Finished Training  10/05/2021 20:21:24
Avg per epoch ptime: 27.14, total 100 epochs ptime: 2713.96


### GoogLeNet Training

In [ ]:
# Load and train GoogLeNet
net, optimizer, scheduler = LoadGLNet(device)
TrainGLNet(net, optimizer, scheduler, criterion, epochs, train_loader)
torch.save(net.state_dict(), model_dir + '/GoogLeNet_CIFAR10_model.pth')
del net
del optimizer
del scheduler

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
/usr/local/lib/python3.7/dist-packages/torchvision/models/googlenet.py:79: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)
  1%|          | 1/157 [00:00<00:20,  7.51it/s]

Adjusting learning rate of group 0 to 1.0000e-02.

Training GoogLeNet
Training starting  10/05/2021 20:23:13
Epoch 1 starting at: 20:23:13


100%|██████████| 40/40 [00:04<00:00,  9.44it/s]
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  1%|          | 1/157 [00:00<00:22,  7.06it/s]

Adjusting learning rate of group 0 to 1.0000e-02.

Epoch 1 of 100, 24.99 s, with average training loss: 1.77926491, and validation loss: nan, validation accuracy: 45.90
Epoch 2 starting at: 20:23:38


  1%|          | 1/157 [00:00<00:21,  7.23it/s]

...

Adjusting learning rate of group 0 to 6.1271e-03.

Epoch 98 of 100, 27.09 s, with average training loss: 0.00165373, and validation loss: nan, validation accuracy: 86.06
Epoch 99 starting at: 21:04:31


  1%|          | 1/157 [00:00<00:22,  6.89it/s]

Adjusting learning rate of group 0 to 6.1271e-03.

Epoch 99 of 100, 26.89 s, with average training loss: 0.00227215, and validation loss: nan, validation accuracy: 85.55
Epoch 100 starting at: 21:04:58


100%|██████████| 40/40 [00:04<00:00,  9.25it/s]


Adjusting learning rate of group 0 to 6.1271e-03.

Epoch 100 of 100, 26.52 s, with average training loss: 0.00802837, and validation loss: nan, validation accuracy: 85.05
Finished Training  10/05/2021 21:05:25
Avg per epoch ptime: 25.32, total 100 epochs ptime: 2532.01


### Save training results

In [ ]:
# Show training results
show_train_hist(train_hist, save=True, path=save_dir + '/CIFAR10_train_hist.png')

# Add zeros at the beginning of data for graph 'smoothness'
train_hist['VGG_accuracy'].insert(0, 0)
train_hist['ResNet_accuracy'].insert(0, 0)
train_hist['GGL_Net_accuracy'].insert(0, 0)
# Show accuracy results
show_accuracy_plot(train_hist, save=True, path=save_dir + '/CIFAR10_accuracy_hist.png')

# Model testing